In [1]:
from pathlib import Path
import rasterio
import numpy as np
import pandas as pd
import geopandas as gpd
from rasterstats import zonal_stats
from datetime import datetime, timedelta
from tqdm import tqdm


In [9]:
import os
os.chdir("..")   # поднимаемся из notebooks в корень проекта
os.getcwd()


'C:\\Users\\i-ryb\\Desktop\\fire_forecast_kurgan'

In [10]:
GRID_PATH = "data_processed/grid_with_y_kurgan.geojson"
grid = gpd.read_file(GRID_PATH)

assert "cell_id" in grid.columns


In [11]:
def process_modis_hdf(hdf_path: Path, grid: gpd.GeoDataFrame) -> pd.DataFrame:
    ndvi_layer = (
        "HDF4_EOS:EOS_GRID:"
        + str(hdf_path)
        + ":MODIS_Grid_16DAY_250m_500m_VI:250m 16 days NDVI"
    )

    with rasterio.open(ndvi_layer) as src:
        ndvi = src.read(1).astype("float32")
        transform = src.transform
        crs = src.crs

    # очистка и масштабирование
    ndvi[ndvi <= -3000] = np.nan
    ndvi = ndvi * 0.0001

    grid_proj = grid.to_crs(crs)

    stats = zonal_stats(
        grid_proj,
        ndvi,
        affine=transform,
        stats=["mean"],
        nodata=np.nan
    )

    # дата из имени файла
    fname = hdf_path.name
    year = int(fname[9:13])
    doy = int(fname[13:16])
    date = datetime(year, 1, 1) + timedelta(days=doy - 1)

    return pd.DataFrame({
        "cell_id": grid["cell_id"].values,
        "date": date,
        "ndvi_mean": [s["mean"] for s in stats]
    })


In [12]:
test_file = next(Path("data_raw/modis_ndvi").rglob("*.hdf"))
test_df = process_modis_hdf(test_file, grid)

test_df.head(), test_df.ndvi_mean.describe()


RasterioIOError: 'HDF4_EOS:EOS_GRID:data_raw\modis_ndvi\2019\MOD13Q1.A2019129.h21v03.061.2020294164717.hdf:MODIS_Grid_16DAY_250m_500m_VI:250m 16 days NDVI' not recognized as being in a supported file format. It could have been recognized by driver HDF4Image, but plugin gdal_HDF4.dll is not available in your installation. You may install it with 'conda install libgdal-hdf4'